In [1]:
!ls -l

total 20800
-rw-r--r--   1 johnnylu  staff    39706 Oct 21 23:50 Classify Structured Data with Feature Columns_TF.ipynb
drwxr-xr-x   4 johnnylu  staff      128 Apr  7  2020 ForinaWine
drwxr-xr-x  11 johnnylu  staff      352 Mar  3  2020 KDD99
drwxr-xr-x   4 johnnylu  staff      128 Mar  3  2020 NSL-KDD
drwxr-xr-x   6 johnnylu  staff      192 Mar  3  2020 UNSW-NB15
-rw-r--r--@  1 johnnylu  staff  5277365 Apr 14  2020 adult-all.csv
-rw-r--r--@  1 johnnylu  staff  5326368 Apr 21  2020 adult_edited.csv


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
df = pd.read_csv('adult_edited.csv', na_values="?")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [5]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [6]:
df['target'] = np.where(df['income']=='>50K', 1, 0)
df.drop(['income'], axis=1, inplace=True)

In [7]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,target
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  target           48842 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 5.6+ MB


In [9]:
from sklearn.impute import SimpleImputer

class DataCleaner:
    """Doc String Placeholder
    
    Usage: 
    >>> DataFrame = DataCleaner(DataFrame).imputer(strategy=('most_frequent', 'median'))
    >>> DataFrame = DataCleaner(DataFrame).outlierClean()
    """
    
    def __init__(self, DataFrame, CATEGORICAL_COLUMNS=None, NUMERICAL_COLUMNS=None):
        self.DataFrame = DataFrame
        if CATEGORICAL_COLUMNS:
            self.CATEGORICAL_COLUMNS = CATEGORICAL_COLUMNS
        else:
            self.CATEGORICAL_COLUMNS = [column for column in self.DataFrame.columns 
                                        if self.DataFrame[column].dtype == 'object']
        if NUMERICAL_COLUMNS:
            self.NUMERICAL_COLUMNS = NUMERICAL_COLUMNS
        else:
            self.NUMERICAL_COLUMNS = [column for column in self.DataFrame.columns
                                      if self.DataFrame[column].dtype in ['int64', 'float64']]
    
    def imputer(self, strategy):
        self.DataFrame[self.CATEGORICAL_COLUMNS] = \
            SimpleImputer(strategy=strategy[0]).fit_transform(self.DataFrame[self.CATEGORICAL_COLUMNS])
        
        self.DataFrame[self.NUMERICAL_COLUMNS] = \
            SimpleImputer(strategy=strategy[1]).fit_transform(self.DataFrame[self.NUMERICAL_COLUMNS])
        
        return self.DataFrame
    
    def outlierClean(self):
        pass
    
    

In [10]:
df = DataCleaner(df).imputer(strategy=('most_frequent', 'median'))

In [11]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,target
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,0.0
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,0.0
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,1.0
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,1.0
4,18.0,Private,103497.0,Some-college,10.0,Never-married,Prof-specialty,Own-child,White,Female,0.0,0.0,30.0,United-States,0.0


In [12]:
df.isnull().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
target             0
dtype: int64

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [14]:
df[['capital-gain', 'hours-per-week']].describe()

,capital-gain,hours-per-week
count,48842.000000,48842.000000
mean,1079.067626,40.422382
std,7452.019058,12.391444
min,0.000000,1.000000
25%,0.000000,40.000000
50%,0.000000,40.000000
75%,0.000000,45.000000
max,99999.000000,99.000000


In [15]:
stat = df['capital-gain'].describe()
IQR = stat['75%'] - stat['25%']
print(IQR)

0.0


In [16]:
df['capital-gain'].unique()

array([    0.,  7688.,  3103.,  6418.,  7298.,  3908., 14084.,  5178.,
       15024., 99999.,  2597.,  2907.,  4650.,  6497.,  1055.,  5013.,
       27828.,  4934.,  4064.,  3674.,  2174., 10605.,  3418.,   114.,
        2580.,  3411.,  4508.,  4386.,  8614., 13550.,  6849.,  2463.,
        3137.,  2885.,  2964.,  1471., 10566.,  2354.,  1424.,  1455.,
        3325.,  4416., 25236.,   594.,  2105.,  4787.,  2829.,   401.,
        4865.,  1264.,  1506., 10520.,  3464.,  2653., 20051.,  4101.,
        1797.,  2407.,  3471.,  1086.,  1848., 14344.,  1151.,  2993.,
        2290., 15020.,  9386.,  2202.,  3818.,  2176.,  5455., 11678.,
        7978.,  7262.,  6514., 41310.,  3456.,  7430.,  2414.,  2062.,
       34095.,  1831.,  6723.,  5060., 15831.,  2977.,  2346.,  3273.,
        2329.,  9562.,  2635.,  4931.,  1731.,  6097.,   914.,  7896.,
        5556.,  1409.,  3781.,  3942.,  2538.,  3887., 25124.,  7443.,
        5721.,  1173.,  4687.,  6612.,  6767.,  2961.,   991.,  2036.,
      

In [17]:
df['hours-per-week'].unique()

array([40., 50., 30., 32., 10., 39., 35., 48., 25., 20., 45., 47.,  6.,
       43., 90., 54., 60., 38., 36., 18., 24., 44., 56., 28., 16., 41.,
       22., 55., 14., 33., 37.,  8., 12., 70., 15., 75., 52., 84., 42.,
       80., 68., 99., 65.,  5., 17., 72., 53., 29., 96., 21., 46.,  3.,
        1., 23., 49., 67., 76.,  7.,  2., 58., 26., 34.,  4., 51., 78.,
       63., 31., 92., 77., 27., 85., 13., 19., 98., 62., 66., 57., 11.,
       86., 59.,  9., 64., 73., 61., 88., 79., 89., 74., 69., 87., 97.,
       94., 82., 91., 81., 95.])